### Outline
#### Goal is to train spaCy NER from litbank data 

- Load annotation data from LitBank
- Create train and validation sets
- Identify entities in text using Matcher (note missed ents in val set, not learning, just matching)
- Train NER from scratch using only language object
- Train NER from scratch  for small en model  
- Fine-tune existing NER pipeline
- Assess results for various approaches 
- Where do we see improvement? When is the model sufficiently useful in research?


In [5]:
!pip install spacy sklearn tqdm
!git clone https://github.com/dbamman/litbank.git
import spacy 
print(f'Using spaCy version {spacy.__version__}')

fatal: destination path 'litbank' already exists and is not an empty directory.
Using spaCy version 3.2.1


In [6]:
from pathlib import Path
entities_path = Path.cwd() / 'litbank' / 'entities' / 'brat'

text_files = [f for f in entities_path.iterdir() if f.suffix == '.txt']
assert len(text_files) == 100
print(f'[*] imported {len(text_files)} files')

[*] imported 100 files


In [7]:
# for each file, create a Doc object and add the annotation data to doc.ents
# our output is a list of Doc objects 
import spacy 
from tqdm.notebook import tqdm
from spacy.tokens import Span, DocBin
from spacy.util import filter_spans


docs = []

#note: not using pretrained model because it adds predictions, just want LitBank data
nlp = spacy.blank("en")

for text_file in tqdm(text_files):
    doc = nlp(text_file.read_text())
    annotation_file = (entities_path / (text_file.stem +'.ann'))
    annotations = annotation_file.read_text().split('\n')
    ents = []
    for annotation in annotations[:-1]:
        label, start, end = annotation.split('\t')[1].split()
        span = doc.char_span(int(start), int(end), label=label)
        if span: # when start and end do not match a valid string, spaCy returns a NoneType span
            ents.append(span)
    
    filtered = filter_spans(ents)
    doc.ents = filtered
    docs.append(doc)
    

assert len(docs) == 100

  0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
# Split the data into sets for training and validation 
from sklearn.model_selection import train_test_split

train_set, validation_set = train_test_split(docs, test_size=0.2)
print(f'Created {len(train_set)} training docs')
print(f'Created {len(validation_set)} validation docs')

Created 80 training docs
Created 20 validation docs


In [9]:
# Add training Docs to DocBin and store to disk
from spacy.tokens import DocBin

# the DocBin will store the training documents
train_db = DocBin()
for doc in train_set:
    train_db.add(doc)
train_db.to_disk("./train.spacy")

In [10]:
# Save the validation Docs to disk 
validation_db = DocBin()
for doc in validation_set:
    validation_db.add(doc)
    
validation_db.to_disk("./dev.spacy") # the spaCy doc refer to development data rather than validation, change our language?

In [11]:
!ls -al train.spacy dev.spacy

-rw-rw-r-- 1 ds ds  335032 Dec 15 09:35 dev.spacy
-rw-rw-r-- 1 ds ds 1240455 Dec 15 09:35 train.spacy


In [14]:
!python3 -m spacy init config ./config.cfg --lang en --pipeline ner

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [13]:
# inspect the new config.cfg file 
!cat config.cfg

[paths]
train = null
dev = null
vectors = null
init_tok2vec = null

[system]
gpu_allocator = null
seed = 0

[nlp]
lang = "en"
pipeline = ["tok2vec","ner"]
batch_size = 1000
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}
upstream = "*"

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${compon

In [15]:
!python3 -m spacy train config.cfg --output ./output --paths.train train.spacy --paths.dev dev.spacy

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-12-15 09:36:48,089] [INFO] Set up nlp object from config
[2021-12-15 09:36:48,098] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-12-15 09:36:48,101] [INFO] Created vocabulary
[2021-12-15 09:36:48,101] [INFO] Finished initializing nlp object
[2021-12-15 09:36:53,628] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00   1117.31    0.15    1.26    0.08    0.00
  2     200      10395.68  57296.75   43.04   59.15   33.83    0.43
  5     400      15726.24  27332.29   47.85   58.17   40.64    0.48
  7     

In [40]:
new_nlp = spacy.load("output/model-last")
for doc in validation_set:
    litbank_ents = doc.ents
    litbank_ents = [e.text for e in litbank_ents]
    new_ents = new_nlp(doc.text).ents
    new_ents = [e.text for e in new_ents]


In [50]:
import random
from spacy import displacy 

new_nlp = spacy.load("output/model-last")
val_doc = random.choice(validation_set)
doc = new_nlp(val_doc.text)

displacy.render(doc, jupyter=True, style="ent")

In [51]:
displacy.render(val_doc, jupyter=True, style="ent")

In [43]:
len(difference)

34

In [47]:
(135 -34) / 135

0.7481481481481481